In [37]:
import pandas as pd

df = pd.read_excel('IN.xlsx')


,Indiana Medicaid Statewide Uniform Preferred Drug List (SUPDL),Unnamed: 1,Unnamed: 2
0,OptumRx Call Center\nFor prior authorization r...,NaN,NaN
1,Indiana Health Coverage Programs (IHCP) Drug C...,NaN,NaN
2,Effective for FFS claims submitted on or after...,NaN,NaN
3, Prior authorization is typically requir...,NaN,NaN
4,Prior authorization request forms are located ...,NaN,NaN
5,Effective for FFS claims submitted on or after...,NaN,NaN
6,Indiana Medicaid Statewide Uniform Preferred D...,NaN,NaN
7,Indiana Health Coverage Programs\n(IHCP) Drug ...,Angiotensin Receptor Blockers ...,DPP4 Inhibitors and Combination Agents ...
8,NaN,NaN,NaN
9,Effective for FFS claims submitted on or after...,NaN,NaN


In [31]:
from docx import Document
import pandas as pd

def extract_bullets_from_tables(docx_path):
    doc = Document(docx_path)
    bullets = []

    # Loop over every table in the document
    for t_idx, table in enumerate(doc.tables):
        # Loop over every row in the table
        for r_idx, row in enumerate(table.rows):
            # Loop over every cell in the row
            for c_idx, cell in enumerate(row.cells):
                # Loop over every paragraph in the cell
                for para in cell.paragraphs:
                    p_pr = para._p.pPr   # the low-level paragraph properties
                    # Detect via <w:numPr> (true for both bullets & numbered lists)
                    if p_pr is not None and p_pr.numPr is not None:
                        text = para.text.strip()
                        if text:
                            bullets.append({
                                "table_index": t_idx,
                                "row": r_idx,
                                "col": c_idx,
                                "text": text
                            })
    return bullets

bullets = extract_bullets_from_tables("IN.docx")
# for item in bullets:
#     print(f"Table {item['table_index']}  (row {item['row']}, col {item['col']}):  • {item['text']}")

df = pd.DataFrame(bullets)
col1 = df[df['col'] == 1]['text'].dropna()
col3 = df[df['col'] == 3]['text'].dropna()

df2 = pd.concat([col1, col3], axis=1).reset_index(drop=True)
df2.columns = ['col1', 'col3']

preferred = df2['col1'].str.split(' o ').str[0].str.strip().unique().astype(str).tolist()
preferred = [i for i in preferred if not i.startswith("QL")]

nonpreferred = df2['col3'].str.split(' o ').str[0].str.strip().unique().astype(str).tolist()
nonpreferred = [i for i in nonpreferred if not i.startswith("QL")]


In [36]:
final_df = pd.DataFrame()
final_df['products'] = preferred + nonpreferred
final_df['Preferred Status'] = ['Preferred' if i in preferred else 'Non-Preferred' for i in final_df['products']]
final_df[final_df['Preferred Status'] == 'Preferred'].head(50)


,products,Preferred Status
0,acyclovir,Preferred
1,valacyclovir,Preferred
2,amantadine,Preferred
3,oseltamivir,Preferred
4,Relenza,Preferred
5,rimantadine,Preferred
6,Paxlovid,Preferred
7,cefdinir,Preferred
8,cefpodoxime,Preferred
9,ciprofloxacin,Preferred
